# Curate an instruction dataset for LLM supervised fine-tuning


In this tutorial, we will INSERT_INTRODUCTION. It will walk you through the following steps:

- INSERT_STEPS. INSERT EMOJI

INSERT_IMAGE
<!-- <img src="../../_static/tutorials/deploying-texttokenclassification-fastapi/deploying-texttokenclassification-fastapi.png" alt="Transformers Log Demo" style="width: 1100px;"> -->

Intro: importance of cleaning public datasets, especially if these have been generated by other models. In this tutorial, you will learn how to build an instruction dataset for your fine-tuning projects by cleaning the Dolly dataset.

Step 1: get Dolly dataset from HF hub. ‘En’ split.

Step 2: Configure questions and write some annotation guidelines

**Questions:**

rating “Does this record need a correction?” 0=no, 1=yes

Text field “correct Instruction” 

text field “correct Input” 

text field “correct response” 

Step 3: turn the dataset into Argilla records with the desired format. In this case:

Record fields**:**

One instruction

One input (context)

One response

Step 4: No overlap → divide the dataset and place in personal workspaces

in this case we don’t need overlap. We only want 1 result per record and we’ll assume that the human annotations have the desired quality. This is for demonstration data.

Tip: for extra quality assurance, you can make a new dataset where annotators assess the quality of the human annotated dataset.

Step 5: Annotate 

- copy fields that need corrections. Empty field = no corrections needed.

Step 6:  Collect annotations 

Step 7 (optional): Publish the dataset

Step 8: fine-tune your model (??)

## Introduction

INSERT_INTRODUCTION

INSERT_POTENTIAL_REFERENCES_TO_OTHER_PART_OF_DOCS

Let's get started!

## Running Argilla

For this tutorial, you will need to have an Argilla server running. There are two main options for deploying and running Argilla:

1. [Deploy Argilla on Hugging Face Spaces](https://huggingface.co/docs/hub/spaces-sdks-docker-argilla): This is the fastest option and the recommended choice for connecting to external notebooks (e.g., Google Colab) if you have an account on Hugging Face.

2. [Launch Argilla using Argilla's quickstart Docker image](../../getting_started/quickstart.ipynb): This is the recommended option if you want Argilla running on your local machine. Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

Tip
    
This tutorial is a Jupyter Notebook. There are two options to run it:

- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab. Don't forget to change the runtime type to GPU for faster model training and inference.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter notebook tool of your choice.
</div>

## Setup

For this tutorial, you'll need to install the Argilla client and a few third party libraries using `pip`:

In [ ]:
%pip install argilla INSERT_PACKAGE==INSERT_VERSION -qqq

: 

Let's import the Argilla module for reading and writing data:

In [ ]:
import argilla as rg

If you are running Argilla using the Docker quickstart image or Hugging Face Spaces, you need to init the Argilla client with the `URL` and `API_KEY`:

In [ ]:
# Replace api_url with the url to your HF Spaces URL if using Spaces
# Replace api_key if you configured a custom API key
rg.init(
    api_url="http://localhost:6900", 
    api_key="admin.apikey"
)

Finally, let's include the imports we need:

In [ ]:
# import bla bla bla with specific version

## First section

## Second section

## Summary

In this tutorial, we learned INSERT_SUMMARY.
This can INSERT_REASON.